# RESUMEN SESIÓN 1: Asincronía y Consumo de APIs Externas

**Cheat-sheet de referencia rápida | FastAPI + Asincronía + httpx**

Este notebook contiene snippets ejecutables y tablas de referencia para consulta rápida.

## TABLA DE CONTENIDOS

1. [Sintaxis Básica Asíncrona](#sintaxis-básica-asíncrona)
2. [Comparativa Sync vs Async](#comparativa-sync-vs-async)
3. [Endpoints FastAPI](#endpoints-fastapi)
4. [Validación con Pydantic](#validación-con-pydantic)
5. [Manejo de Errores](#manejo-de-errores)
6. [Consumo de APIs con httpx](#consumo-de-apis-con-httpx)
7. [BackgroundTasks](#backgroundtasks)
8. [Patrones con asyncio.gather()](#patrones-con-asynciogather)
9. [Códigos de Estado HTTP](#códigos-de-estado-http)
10. [Timeouts Recomendados](#timeouts-recomendados)

---

## SINTAXIS BÁSICA ASÍNCRONA

### Definir función asíncrona

In [ ]:
import asyncio

# Función asíncrona (corutina)
async def mi_funcion():
    await asyncio.sleep(1)
    return "completado"

# Ejecutar en Jupyter/IPython (ya hay event loop)
resultado = await mi_funcion()
print(resultado)

### Ejecutar múltiples corutinas en paralelo

In [ ]:
# Todas se ejecutan simultáneamente
async def tarea(nombre, segundos):
    await asyncio.sleep(segundos)
    return f"{nombre} completada"

resultados = await asyncio.gather(
    tarea("A", 1),
    tarea("B", 2),
    tarea("C", 1)
)

print(resultados)

---

## COMPARATIVA SYNC VS ASYNC

| Aspecto | **Síncrono** | **Asíncrono** |
|---------|--------------|---------------|
| **Sintaxis** | `def funcion()` | `async def funcion()` |
| **Espera** | `time.sleep(1)` | `await asyncio.sleep(1)` |
| **Bloqueo** | Bloquea TODO el programa | Solo suspende la corutina |
| **Concurrencia** | Secuencial (una tras otra) | Paralelo (varias a la vez) |
| **Rendimiento I/O** | Malo (espera ociosa) | Excelente (aprovecha esperas) |
| **Uso ideal** | CPU-bound (cálculos) | I/O-bound (red, DB, archivos) |

### Ejemplo visual: 3 tareas de 1 segundo cada una

In [ ]:
import time

# SÍNCRONO: 3 segundos totales
def tarea_sync():
    time.sleep(1)
    return "completada"

print("Ejecución SÍNCRONA:")
inicio = time.time()
for i in range(3):
    tarea_sync()
print(f"Tiempo: {time.time() - inicio:.2f}s (esperado: ~3s)")

In [ ]:
# ASÍNCRONO: 1 segundo total
async def tarea_async():
    await asyncio.sleep(1)
    return "completada"

print("Ejecución ASÍNCRONA:")
inicio = time.time()
await asyncio.gather(
    tarea_async(),
    tarea_async(),
    tarea_async()
)
print(f"Tiempo: {time.time() - inicio:.2f}s (esperado: ~1s)")
print("\nMejora: 3x más rápido")

---

## ENDPOINTS FASTAPI

### Endpoint síncrono (bloqueante)

In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/productos")
def listar_productos():  # def normal
    return [{"id": 1, "nombre": "Laptop"}]

### Endpoint asíncrono (no bloqueante)

In [ ]:
@app.get("/productos-async")
async def listar_productos_async():  # async def
    await asyncio.sleep(0.5)  # Simular consulta DB
    return [{"id": 1, "nombre": "Laptop"}]

### Endpoint con path parameter

In [ ]:
@app.get("/productos/{producto_id}")
async def obtener_producto(producto_id: int):
    return {"id": producto_id, "nombre": "Laptop"}

### Endpoint POST con body

In [ ]:
from pydantic import BaseModel

class ProductoCreate(BaseModel):
    nombre: str
    precio: float

@app.post("/productos")
async def crear_producto(producto: ProductoCreate):
    return {"id": 1, **producto.model_dump()}

---

## VALIDACIÓN CON PYDANTIC

### Modelo básico con Field()

In [ ]:
from pydantic import BaseModel, Field

class Usuario(BaseModel):
    nombre: str = Field(..., min_length=3)
    edad: int = Field(..., ge=18, le=100)
    email: str

# Prueba
usuario = Usuario(nombre="Ana", edad=25, email="ana@ejemplo.com")
print(usuario.model_dump())

### Validador personalizado (Pydantic v2)

In [ ]:
from pydantic import field_validator

class UsuarioConValidacion(BaseModel):
    email: str
    
    @field_validator('email')
    @classmethod
    def validar_email(cls, v: str) -> str:
        if '@' not in v:
            raise ValueError('Email inválido')
        return v.lower()

# Prueba válida
usuario = UsuarioConValidacion(email="ANA@EJEMPLO.COM")
print(f"Email normalizado: {usuario.email}")

# Prueba inválida
try:
    usuario_malo = UsuarioConValidacion(email="sin-arroba")
except Exception as e:
    print(f"Error esperado: {e}")

### Validadores comunes con Field()

| Tipo | Validación | Ejemplo |
|------|------------|----------|
| String | Longitud mínima/máxima | `Field(..., min_length=3, max_length=50)` |
| Integer | Mayor o igual (>=) | `Field(..., ge=18)` |
| Integer | Menor o igual (<=) | `Field(..., le=100)` |
| Float | Mayor que (>) | `Field(..., gt=0)` |
| URL | Validación automática | `HttpUrl` |
| Lista | Valor por defecto | `Field(default_factory=list)` |

In [ ]:
from pydantic import HttpUrl

class ModeloCompleto(BaseModel):
    nombre: str = Field(..., min_length=3, max_length=50)
    edad: int = Field(..., ge=18, le=100)
    precio: float = Field(..., gt=0)
    sitio_web: HttpUrl
    tags: list[str] = Field(default_factory=list)

# Prueba
modelo = ModeloCompleto(
    nombre="Ana",
    edad=25,
    precio=99.99,
    sitio_web="https://ejemplo.com",
    tags=["python", "fastapi"]
)
print(modelo.model_dump())

---

## MANEJO DE ERRORES

### Lanzar HTTPException

In [ ]:
from fastapi import HTTPException

# Base de datos simulada
base_datos = {1: {"nombre": "Laptop"}}

@app.get("/items/{item_id}")
async def obtener_item(item_id: int):
    if item_id not in base_datos:
        raise HTTPException(
            status_code=404,
            detail="Producto no encontrado"
        )
    
    return base_datos[item_id]

### Patrón de validación múltiple

In [ ]:
db = {1: {"nombre": "Laptop", "stock": 5}}

@app.get("/productos-validados/{producto_id}")
async def obtener_producto_validado(producto_id: int):
    # Validación 1: ID válido
    if producto_id < 1:
        raise HTTPException(400, "ID inválido")
    
    # Validación 2: Existe en DB
    if producto_id not in db:
        raise HTTPException(404, "No encontrado")
    
    producto = db[producto_id]
    
    # Validación 3: Tiene stock
    if producto['stock'] == 0:
        raise HTTPException(400, "Sin stock")
    
    return producto

---

## CONSUMO DE APIS CON HTTPX

### GET básico

In [ ]:
import httpx

async def obtener_usuario(user_id: int):
    url = f"https://jsonplaceholder.typicode.com/users/{user_id}"
    
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error {response.status_code}")

# Prueba
usuario = await obtener_usuario(1)
print(f"Usuario: {usuario['name']}")

### GET con timeout

In [ ]:
async def obtener_usuario_timeout(user_id: int):
    url = f"https://jsonplaceholder.typicode.com/users/{user_id}"
    
    async with httpx.AsyncClient(timeout=5.0) as client:
        response = await client.get(url)
        return response.json()

# Prueba
usuario = await obtener_usuario_timeout(1)
print(f"Email: {usuario['email']}")

### POST con datos JSON

In [ ]:
async def crear_post(datos: dict):
    url = "https://jsonplaceholder.typicode.com/posts"
    
    async with httpx.AsyncClient() as client:
        response = await client.post(url, json=datos)
        return response.json()

# Prueba
nuevo_post = await crear_post({
    "title": "Mi post",
    "body": "Contenido del post",
    "userId": 1
})
print(f"Post creado con ID: {nuevo_post['id']}")

### Manejo robusto de errores

In [ ]:
async def consultar_api_segura(url: str):
    try:
        async with httpx.AsyncClient(timeout=5.0) as client:
            response = await client.get(url)
            return response.json()
            
    except httpx.TimeoutException:
        return {"error": "Timeout"}
        
    except httpx.RequestError as e:
        return {"error": f"Error de red: {e}"}

# Prueba con URL válida
resultado = await consultar_api_segura("https://jsonplaceholder.typicode.com/posts/1")
print(f"Título: {resultado.get('title', 'Error')}")

# Prueba con URL inválida
resultado = await consultar_api_segura("https://url-inexistente-12345.com")
print(f"Resultado: {resultado}")

### Headers personalizados

In [ ]:
async def consultar_con_auth(url: str, token: str):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    async with httpx.AsyncClient() as client:
        response = await client.get(url, headers=headers)
        return response.json()

# Ejemplo (esta API no requiere auth, solo muestra el patrón)
# resultado = await consultar_con_auth("https://api.com/datos", "mi_token")

---

## BACKGROUNDTASKS

### Función de background simple

In [ ]:
from fastapi import BackgroundTasks
from datetime import datetime

def enviar_email(email: str, mensaje: str):
    """Esta función se ejecuta DESPUÉS de retornar la respuesta"""
    time.sleep(2)  # Simular envío
    print(f"Email enviado a {email}: {mensaje}")

@app.post("/registro")
async def registrar_usuario(email: str, background_tasks: BackgroundTasks):
    # Agregar tarea al background
    background_tasks.add_task(enviar_email, email, "Bienvenido")
    
    # Retorna INMEDIATAMENTE
    return {"status": "registrado"}

### Múltiples background tasks

In [ ]:
def actualizar_inventario(producto_id: int):
    print(f"Inventario actualizado: producto {producto_id}")

def enviar_confirmacion(user_id: int):
    print(f"Confirmación enviada a usuario {user_id}")

def registrar_venta(user_id: int, producto_id: int):
    print(f"Venta registrada: usuario {user_id}, producto {producto_id}")

@app.post("/compra")
async def procesar_compra(
    user_id: int, 
    producto_id: int,
    background_tasks: BackgroundTasks
):
    # Agregar varias tareas
    background_tasks.add_task(actualizar_inventario, producto_id)
    background_tasks.add_task(enviar_confirmacion, user_id)
    background_tasks.add_task(registrar_venta, user_id, producto_id)
    
    return {"status": "procesando"}

### Background task asíncrona

In [ ]:
async def actualizar_metricas_async(user_id: int):
    """También puede ser async def"""
    async with httpx.AsyncClient() as client:
        await client.post(
            "https://analytics.com/evento", 
            json={"user": user_id}
        )

@app.post("/accion")
async def realizar_accion(user_id: int, background_tasks: BackgroundTasks):
    background_tasks.add_task(actualizar_metricas_async, user_id)
    return {"status": "ok"}

---

## PATRONES CON ASYNCIO.GATHER()

### Patrón 1: Múltiples peticiones a misma API

In [ ]:
async def obtener_usuarios_multiples(ids: list[int]):
    """Obtiene varios usuarios en paralelo"""
    
    tareas = [obtener_usuario(user_id) for user_id in ids]
    resultados = await asyncio.gather(*tareas)
    
    return resultados

# Prueba
usuarios = await obtener_usuarios_multiples([1, 2, 3])
print(f"Obtenidos {len(usuarios)} usuarios en paralelo")
for u in usuarios:
    print(f"- {u['name']}")

### Patrón 2: Diferentes fuentes de datos

In [ ]:
async def obtener_posts(user_id: int):
    url = f"https://jsonplaceholder.typicode.com/posts?userId={user_id}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        return response.json()

async def dashboard_usuario(user_id: int):
    """Obtiene datos de múltiples servicios en paralelo"""
    
    perfil, posts = await asyncio.gather(
        obtener_usuario(user_id),
        obtener_posts(user_id)
    )
    
    return {
        "perfil": perfil,
        "total_posts": len(posts)
    }

# Prueba
dashboard = await dashboard_usuario(1)
print(f"Usuario: {dashboard['perfil']['name']}")
print(f"Posts: {dashboard['total_posts']}")

### Patrón 3: Con manejo de errores

In [ ]:
async def obtener_datos_seguros(urls: list[str]):
    """Obtiene datos pero continúa si alguna falla"""
    
    tareas = [consultar_api_segura(url) for url in urls]
    
    # return_exceptions=True evita que una falla detenga todo
    resultados = await asyncio.gather(*tareas, return_exceptions=True)
    
    # Filtrar errores
    datos_validos = [r for r in resultados if not isinstance(r, Exception)]
    
    return datos_validos

# Prueba con una URL válida y una inválida
urls = [
    "https://jsonplaceholder.typicode.com/posts/1",
    "https://url-inexistente.com"
]
resultados = await obtener_datos_seguros(urls)
print(f"Resultados obtenidos: {len(resultados)}")

### Patrón 4: Endpoint FastAPI con gather

In [ ]:
@app.get("/analizar-usuario/{user_id}")
async def analizar_usuario_endpoint(user_id: int):
    """Trae datos de usuario y posts EN PARALELO"""
    
    datos_usuario, posts = await asyncio.gather(
        obtener_usuario(user_id),
        obtener_posts(user_id)
    )
    
    return {
        "usuario": datos_usuario["name"],
        "total_posts": len(posts)
    }

---

## CÓDIGOS DE ESTADO HTTP

### Tabla de referencia rápida

| Código | Nombre | Cuándo usarlo |
|--------|--------|---------------|
| **200** | OK | Petición exitosa (GET, PUT, PATCH) |
| **201** | Created | Recurso creado exitosamente (POST) |
| **204** | No Content | Éxito sin contenido (DELETE) |
| **400** | Bad Request | Datos inválidos del cliente |
| **401** | Unauthorized | Autenticación requerida |
| **403** | Forbidden | Sin permisos (aunque esté autenticado) |
| **404** | Not Found | Recurso no existe |
| **422** | Unprocessable Entity | Validación de Pydantic falló |
| **500** | Internal Server Error | Error del servidor |
| **503** | Service Unavailable | Servicio temporalmente no disponible |

### Ejemplos de uso

In [ ]:
from fastapi import status

# 400 - Datos inválidos
if edad < 18:
    raise HTTPException(
        status_code=status.HTTP_400_BAD_REQUEST,
        detail="Edad debe ser mayor a 18"
    )

# 404 - No encontrado
if user_id not in database:
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail="Usuario no encontrado"
    )

# 401 - No autenticado
if not token:
    raise HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Token requerido"
    )

# 403 - Sin permisos
if user_role != "admin":
    raise HTTPException(
        status_code=status.HTTP_403_FORBIDDEN,
        detail="Solo administradores"
    )

---

## TIMEOUTS RECOMENDADOS

### Tabla de timeouts por tipo de operación

| Tipo de API | Timeout Recomendado | Justificación |
|-------------|---------------------|---------------|
| **APIs internas** (microservicios) | 2-5 segundos | Red local, deberían ser rápidas |
| **APIs externas** (terceros) | 5-10 segundos | Red pública, más latencia |
| **Generación de contenido** (IA, procesamiento) | 30-60 segundos | Operaciones computacionalmente intensivas |
| **Webhooks** (notificaciones salientes) | 3-5 segundos | No esperar respuestas lentas |
| **Health checks** | 1-2 segundos | Deben ser muy rápidos |

### Configuración de timeouts en httpx

In [ ]:
# Timeout simple (aplica a todo)
async with httpx.AsyncClient(timeout=5.0) as client:
    response = await client.get(url)

In [ ]:
# Timeout granular (control fino)
from httpx import Timeout

timeout = Timeout(
    connect=5.0,  # Tiempo para establecer conexión
    read=10.0,    # Tiempo para recibir respuesta
    write=5.0,    # Tiempo para enviar datos
    pool=None     # Sin límite para pool
)

async with httpx.AsyncClient(timeout=timeout) as client:
    response = await client.get(url)

In [ ]:
# Sin timeout (PELIGROSO en producción)
async with httpx.AsyncClient(timeout=None) as client:
    response = await client.get(url)  # Puede esperar PARA SIEMPRE

### Manejo de timeout en endpoint

In [ ]:
@app.get("/datos-externos")
async def obtener_datos_externos():
    try:
        async with httpx.AsyncClient(timeout=5.0) as client:
            response = await client.get("https://api-lenta.com/datos")
            return response.json()
            
    except httpx.TimeoutException:
        raise HTTPException(
            status_code=504,  # Gateway Timeout
            detail="API externa tardó demasiado"
        )

---

## SNIPPETS RÁPIDOS PARA COPIAR/PEGAR

### App FastAPI completa mínima

In [ ]:
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel
import httpx
import asyncio

app = FastAPI(title="Mi API")

class Item(BaseModel):
    nombre: str
    precio: float

@app.get("/")
async def root():
    return {"mensaje": "API funcionando"}

@app.get("/items/{item_id}")
async def obtener_item(item_id: int):
    if item_id < 1:
        raise HTTPException(400, "ID inválido")
    return {"id": item_id, "nombre": "Item"}

@app.post("/items")
async def crear_item(item: Item):
    return {"id": 1, **item.model_dump()}

### TestClient para pruebas

In [ ]:
from fastapi.testclient import TestClient

client = TestClient(app)

# GET
response = client.get("/items/1")
print(response.status_code)  # 200
print(response.json())

# POST
response = client.post("/items", json={"nombre": "Laptop", "precio": 999.99})
print(response.json())

### Cliente httpx reutilizable

In [ ]:
class APIClient:
    def __init__(self, base_url: str, timeout: float = 5.0):
        self.base_url = base_url
        self.timeout = timeout
    
    async def get(self, endpoint: str):
        url = f"{self.base_url}{endpoint}"
        async with httpx.AsyncClient(timeout=self.timeout) as client:
            response = await client.get(url)
            response.raise_for_status()
            return response.json()
    
    async def post(self, endpoint: str, data: dict):
        url = f"{self.base_url}{endpoint}"
        async with httpx.AsyncClient(timeout=self.timeout) as client:
            response = await client.post(url, json=data)
            response.raise_for_status()
            return response.json()

# Uso
api = APIClient("https://jsonplaceholder.typicode.com")
usuarios = await api.get("/users")
print(f"Usuarios obtenidos: {len(usuarios)}")

---

## CHECKLIST DE BUENAS PRÁCTICAS

### Antes de llevar a producción:

- [ ] Todos los endpoints tienen `async def` si hacen I/O
- [ ] Todas las peticiones HTTP tienen timeout configurado
- [ ] Los modelos Pydantic validan correctamente los datos
- [ ] Los errores lanzan HTTPException con códigos apropiados
- [ ] Las operaciones lentas usan BackgroundTasks
- [ ] Las peticiones paralelas usan `asyncio.gather()`
- [ ] Los secrets (API keys) están en variables de entorno
- [ ] Hay logging apropiado (no solo prints)
- [ ] Los endpoints tienen documentación (docstrings)
- [ ] Se probó con TestClient

---

## COMANDOS ÚTILES

### Instalar dependencias

```bash
pip install fastapi==0.115.0 httpx==0.27.0 uvicorn[standard]==0.32.0
```

### Ejecutar servidor de desarrollo

```bash
# Puerto 8000 (default)
uvicorn main:app --reload

# Puerto personalizado
uvicorn main:app --reload --port 8080

# Accesible desde red local
uvicorn main:app --reload --host 0.0.0.0 --port 8000
```

### Documentación automática

Una vez corriendo el servidor:
- **Swagger UI:** `http://localhost:8000/docs`
- **ReDoc:** `http://localhost:8000/redoc`
- **OpenAPI JSON:** `http://localhost:8000/openapi.json`

---

## RECURSOS ADICIONALES

### Documentación oficial:

- **FastAPI:** https://fastapi.tiangolo.com/
- **Pydantic:** https://docs.pydantic.dev/
- **httpx:** https://www.python-httpx.org/
- **asyncio:** https://docs.python.org/3/library/asyncio.html

### Herramientas útiles:

- **httpie:** Cliente HTTP para terminal (`pip install httpie`)
- **Postman:** Cliente HTTP con interfaz gráfica
- **pytest:** Testing framework (`pip install pytest`)
- **pytest-asyncio:** Plugin para tests asíncronos

---

**Fin del resumen | Sesión 1: Asincronía y Consumo de APIs Externas**

Este notebook contiene todos los snippets necesarios para trabajar con FastAPI de forma asíncrona. Puedes ejecutar cualquier celda de código directamente para probar los ejemplos.